In [ ]:
import requests
import json
import pandas
import SciServer.CasJobs as CasJobs

In [ ]:
with open('/home/idies/keystone.token', 'r') as f:
    token = f.read().rstrip('\n')
token

In [ ]:
sqURL="http://scitest02.sdss.pha.jhu.edu/skyquery/Api/V1/"
queuesURL=sqURL+"Jobs.svc/queues"
jobURL=sqURL+"/Jobs.svc/jobs/" 

In [ ]:
def queues(token):
    headers={'X-Auth-Token': token,'Content-Type': 'application/json','Accept':'application:json'}

    rsp=requests.get(queuesURL,headers=headers)
    if rsp.status_code ==200:
        return(rsp.json())
    else:
        return(rsp.text)

In [ ]:
def jobsurl(queue):
    return queuesURL+"/"+queue+"/jobs"

def jobs(token, queue='Quick'):
    headers = {'Content-Type': 'application/json','Accept': 'application/json'}
    headers['X-Auth-Token']=  token
    rsp=requests.get(jobsurl(queue), headers=headers)
    return (rsp.json())

In [ ]:
def jobstatus(guid,token):
    headers = {'Content-Type': 'application/json','Accept': 'application/json'}
    headers['X-Auth-Token']=  token
    rsp=requests.get(jobURL+guid, headers=headers)
    return (rsp.json())

In [ ]:
def submit(query,token, queue="Quick"):
    headers = {'Content-Type': 'application/json','Accept': 'application/json'}
    headers['X-Auth-Token']=  token
    body={"queryJob":{"query":query}}
    data=json.dumps(body).encode()
    postResponse = requests.post(jobsurl(queue),data=data,headers=headers)    
    if postResponse.status_code == 200:
        r=postResponse.json()
        return(r['queryJob']['guid'])
    else:
        raise NameError(postResponse.json())

In [ ]:
jobid=submit("SELECT TOP 100 objid, ra, dec, u, g, r, i, z FROM SDSSDR7:PhotoObjAll",token)

In [ ]:
jobstatus(jobid,token)
# run a few times until done. result stored in MyDB.QuickResults.

In [ ]:
# retrieve using casjobs
sql="select * from quickresults"
r=CasJobs.executeQuery(sql,"MyDB", token=token)
df=pandas.read_csv(r,index_col=None)
df